In [1]:
%load_ext autoreload
%autoreload 2   

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import asyncio
from loguru import logger
import os
import matplotlib.pyplot as plt
import time
import qscope.server
import qscope.system
import qscope.types
import qscope.util
from qscope.scripting import (
    meas_close_after_nsweeps, 
    meas_stop_after_nsweeps,
    meas_wait_for_nsweeps,
)

In [ ]:
# Save Name
SAVE_DIR = 'C:\\ExperimentalData\\2025\\2025-04\\2025-04-11_testing\\'
SAVE_NAME = 'esr_sweep'

Using the custom tag: ( testing ) for the data directory.
Connected to Camera: ZYLA-5.5-USB3-W
Connected to rf source: windfreak


In [ ]:
# Open the server or connect to an existing one

# Start the client server
qscope.server.start_client_log(log_to_stdout=True, log_level="INFO")

# Start the connection manager
manager = qscope.server.ConnectionManager()
# Start connection manager and define the system
try:
    manager.start_local_server(
        "zyla",
        # "hqdm",
        # "gmx",
        # "mock",
    ) # logs go to ~./qscope/server.log
    # Connect to the server
    manager.connect()
    manager.startup()
except qscope.server.ServerAlreadyRunning:
    manager.connect()
    pass

In [ ]:
# Define the frequency sweep that will be taken
# f_list = qscope.util.gen_linear_sweep_list(2770, 2970, 100)
f_list = qscope.util.gen_centred_sweep_list(2988.4, 10, 101)

In [ ]:
# Configure the measurement
config = qscope.types.SGAndorCWESRConfig(
    sweep_x=f_list,
    ref_mode="no_rf",
    exposure_time=15e-3,
    frame_shape=(256, 256),
    hardware_binning=(1, 1),
    avg_per_point=4,
    fmod_freq=0,
    rf_pow=-40.0,
    laser_delay=0.0,
)


In [ ]:
# Take a single measurement 
NUM_SWEEPS = 1
meas_id = manager.add_measurement(config)
manager.start_measurement_wait(meas_id)
sweep_data = meas_stop_after_nsweeps(manager, meas_id, NUM_SWEEPS, timeout=60)
manager.close_measurement_wait(meas_id)


Setting the roi to:  (1152, 1408, 952, 1208)


In [ ]:
# Plot the raw data and the normalized data

plt.figure(figsize=(10, 6))
plt.subplot(2, 1, 1)
plt.plot(f_list, sweep_data[0], label='signal')
plt.plot(f_list, sweep_data[1], label='reference')
plt.xlabel('Frequency (MHz)')
plt.ylabel('Intensity (a.u.)')
plt.title('ESR Sweep')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(f_list, sweep_data[0] / sweep_data[1], label='normalized signal')
plt.xlabel('Frequency (MHz)')
plt.ylabel('Normalized Intensity (a.u.)')
plt.title('Normalized ESR Sweep')
plt.legend()

plt.tight_layout()
plt.show()


# Now sweep a parameter

In [ ]:
# Let's sweep the rf power and take a measurement at each power level
rf_pow_list = np.linspace(-40, -20, 11)
NUM_SWEEPS = 1
param_sweep_data = np.zeros((len(rf_pow_list), 2, len(f_list)), dtype=np.float32)

i = 0
for rf_pow in rf_pow_list:
    # Configure the measurement
    config = qscope.types.SGAndorCWESRConfig(
        sweep_x=f_list,
        ref_mode="no_rf",
        exposure_time=15e-3,
        frame_shape=(256, 256),
        hardware_binning=(1, 1),
        avg_per_point=4,
        fmod_freq=0,
        rf_pow=rf_pow,
        laser_delay=0.0,
    )

    # Take a single measurement 
    meas_id = manager.add_measurement(config)
    manager.start_measurement_wait(meas_id)
    sweep_data = meas_stop_after_nsweeps(manager, meas_id, NUM_SWEEPS, timeout=60)
    manager.close_measurement_wait(meas_id)

    # Append the data to the list
    param_sweep_data[i,::] = sweep_data
    i += 1
    


In [ ]:
# Plot the normalized data
plt.figure(figsize=(10, 6))
plt.imshow(param_sweep_data[:, 0, :] / param_sweep_data[:, 1, :], 
           aspect='auto', 
           extent=[f_list[0], f_list[-1], rf_pow_list[0], rf_pow_list[-1]], 
           origin='lower')
plt.colorbar(label='Normalized Intensity (a.u.)')
plt.xlabel('Frequency (MHz)')
plt.ylabel('RF Power (dBm)')
plt.title('Normalized ESR Sweep')
plt.tight_layout()
plt.show()

In [ ]:
# Save the data to a file
np.savez(os.path.join(SAVE_DIR, SAVE_NAME), 
         f_list=f_list, 
         rf_pow_list=rf_pow_list, 
         param_sweep_data=param_sweep_data)


In [ ]:
# close the connection manager
manager.close()
